In [1]:
import pandas as pd
import airportsdata
import csv
import codecs
import urllib.request
import urllib.error
import sys
import datetime as dt

cleaned_df = pd.read_csv('/Users/favea/Downloads/swiss-data/10122022b_cleaned.csv', index_col=0)

In [2]:
airports = airportsdata.load('IATA') # key is the ICAO identifier (the default)

In [3]:
import json

def get_data_weather(row):
    
    weather_api_endpoint = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?'
    query_params = '&contentType=json&aggregateMinutes=15&unitGroup=metric&includeAstronomy=false&collectStationContributions=false&key={}&startDateTime={}&endDateTime={}&locations={},{}'
    ApiKey='6SG4URSNKE3MLQK23A4E7R43L'
    latitude, longitude = airports[row.origin]['lat'], airports[row.origin]['lon']
    date = row.scheduled_time_departure
    start = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S').isoformat()
    query_params=query_params.format(ApiKey, start, start,latitude,longitude)

    try:
        response = urllib.request.urlopen(weather_api_endpoint +"?"+ query_params)
        weatherData = json.loads(response.read().decode('utf-8')) 

        locations=weatherData["locations"]
        for locationid in locations:  
            location=locations[locationid]
            for value in location["values"]:


                weather = dict(origin = row.origin, destination = row.destination, scheduled_time_departure = row.scheduled_time_departure,
                               temperature = value["temp"], 
                               precipitation = value["precip"],
                               windSpeed = value["wspd"],
                               dewpoint = value["dew"],
                               humidity = value["humidity"])
                return weather
    except Exception as e:
        print(e)
        return  dict(origin = row.origin, destination = row.destination, scheduled_time_departure = row.scheduled_time_departure,
                                temperature = None, 
                               precipitation = None,
                               windSpeed =None,
                               dewpoint = None,
                               humidity = None)
        
        
    


In [4]:
records = []
START = 2960 # Modify this (You have 1000 records free for day)
END = 3960 # modify this 

for row in cleaned_df.iloc[START:END].itertuples():
    records.append(get_data_weather(row))


In [5]:
weather = pd.DataFrame.from_records(records)

In [6]:
weather.drop_duplicates()

,origin,destination,scheduled_time_departure,temperature,precipitation,windSpeed,dewpoint,humidity
0,ZRH,PVG,2021-09-20 09:30:00,NaN,0.0,NaN,NaN,NaN
1,ZRH,GVA,2021-09-20 05:30:00,11.2,0.0,0.0,9.9,91.70
2,ZRH,LHR,2021-09-20 10:05:00,11.9,0.0,2.2,11.4,96.74
4,ZRH,GVA,2021-09-20 10:45:00,12.0,0.0,1.8,11.0,93.60
6,ZRH,GVA,2021-09-20 15:55:00,16.0,0.0,3.6,11.1,72.23
...,...,...,...,...,...,...,...,...
988,ZRH,EWR,2021-10-25 15:20:00,NaN,0.0,NaN,NaN,NaN
989,ZRH,LAX,2021-10-25 10:55:00,5.1,0.0,3.6,5.0,100.00
992,ZRH,MIA,2021-10-25 11:00:00,5.8,0.0,1.8,5.0,94.34
996,ZRH,GRU,2021-10-25 20:40:00,NaN,0.0,NaN,NaN,NaN


In [7]:
out_csv = f"/Users/favea/Downloads/swiss-data/weather_{START}_{END}.csv"

In [8]:
weather.to_csv(out_csv)

In [9]:
import wandb

with wandb.init(
    project="swiss-delay-prediction", entity=None, job_type="weather-data"
) as run:
    table_merged = wandb.Table(dataframe=weather)

    # Create an artifact for our dataset
    dataset_artifact = wandb.Artifact(
        "dataset-weather",
        type="dataset",
        description="Table containing weather info for observations in cleaned dataframe",
    )
    # Add the table to the artifact & log the artifact
    dataset_artifact.add(table_merged, "data-table-delay-cleaned")
    dataset_artifact.add_file("/Users/favea/Downloads/swiss-data/10122022b_cleaned.csv")

    # Add the
    run.log_artifact(dataset_artifact)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andrea-faviait. Use `wandb login --relogin` to force relogin
